#**Quick processing of data sets for input to model**

**Data Cleaning, Feature Generation**

Andreas Theodoulou and Michael Gaidis (May, 2020)

#**Data Ouput from This Notebook**

##**1.  *shops_new*** </br>
**60 rows**, corresponding to the 60 original shops</br>
**14 columns:**
 * shop_id (categorical; 0-59, int8, from original data set)
 * shop_tested (categorical; bool, indicating if the shop is in test set)
 * shop_type (categorical; string object: online, small shop, mall, SEC, Mega)
 * shop_type_enc (categorical; int8, ordinal/weighted encoding based on number of rows present in test, scaled to cover roughly the same range as shop_id values (0-59))
 * shop_city (categorical; string)
 * shop_city_enc (categorical; int8, encoding weighted like shop_type)
 * shop_federal_district (categorical; string object)
 * shop_federal_district_enc (categorical; int8, encoding weighted like shop_type)
 * s_type_broad (categorical; string object: like shop_type, but fewer categories by merging together "Mall","Mega","SEC")
 * s_type_broad_enc (categorical; int8, ordinal encoding roughly weighted by shop size, 0-60 scale)
 * fd_popdens (categorical; string object: 4 categories named by population density in the shop's federal district)
 * fd_popdens_enc (categorical; int8; ordinal encoding weight based on population density)
 * fd_gdp (categorical; string object: 3 categories named by gdp per person)
 * fd_gdp_enc (categorical; int8; ordinal encoding weight based on gdp/person)

</br>

##**2.  *items_new*** </br>
**22,170 rows**, corresponding to the 22,170 original items</br>
**7 columns:**
 * item_id  (categorical; 0 - 22169, int16, from original data set)
 * item_tested (categorical; bool, indicating if item is in *test* set)
 * item_category_id (categorical; 0 - 83, int8, from original data set)
 * cluster_code (categorical; int32, weighted encoding based on similarity of item names in a given cluster)
 * item_category3 (categorical; string)
 * item_category3_enc (categorical; int8, random nominal encoding done by pandas)
 * item_category4 (categorical; string)
 * item_category4_enc (categorical; int8, random nominal encoding done by pandas)

</br>

##**3.  *sales_train_cleaned*** - remove outliers and adjust shops</br>
**2,914,268 rows**, corresponding to the original rows ***with outliers removed*** (6 outlier rows deleted)</br>
and ***with shops 9 and 13 rows removed*** 
</br>

**6 columns:**
 * date (pd.datetime format)
 * date_block_num (int8) 
 * shop_id (categorical; int8, range **2** to 59, and no entries for 9, 11, 13 either, having removed shops 9 and 13, and merged shops 0,1,11)
 * item_id (categorical; int16, range 0 to 22,169 (but only 21,671 present in *sales_train_cleaned*))
 * item_price (float32; downcast, max is near 60,000)  
 * item_cnt_day (int16, range is roughly 0 to 1000)

</br>

</br>

---
---

##**4a. *sales_train_cln_mrg:***
* remove outliers, 
* adjust shops (remove 9,13, and merge away 0,1,11)
* merge with encoded features from *shops_new* and *items_new* dataframes</br>

**2,914,268 rows,** corresponding the above *sales_train_cleaned* data set </br>
**18 columns,** corresponding to the 6 from *sales_train_clened* plus 12 (encoded) categorical columns coming from *shops_new* and *items_new*</br>

Column descriptions for sales_train_cln_mrg:</br>

| Column Name | DType | Description |
| ----------: | :---: | :--------- |
| date | datetime64 | ordinal day, month, year of transaction in that row |
| date_block_num | int8 | ordinal-encoded month # from start of train data |
| shop_id | int8 | categorical range(60) original shop_id values, minus 0,1,9,11,13 |
| item_id | int16 | categorical range(22170) original item_id values, 21671 present in *sales_train_cln_mrg* |
| item_price | float32 | continuous variable, downcast from float64; price is in range (0 to 59200] |
| item_cnt_day | int16 | continuous variable, items sold during the day of the sales_train row; range = [-22 to 669] |
| shop_tested | bool | True if shop id is in the test set |
| shop_type_enc | int8 | Categorical feature indicating small shop / mall / SEC / online... |
| shop_city_enc | int8 | Categorical feature indicating which city hosts the shop |
| shop_federal_district_enc | int8 | Categorical feature indicating which federal district the shop is in |
| s_type_broad_enc | int8 | Categorical feature like shop_type_enc, but merging together mall/Mega/SEC so fewer categories |
| fd_popdens_enc | int8 | Categorical feature indicating population density of the federal district the shop is in |
| fd_gdp_enc | int8 | Categorical feature indicating gdp/person for the federal district the shop is in |
| item_tested | bool | True if item id is in the test set |
| item_category_id | int8 | Original category codes for the items (0 to 83) |
| cluster_code | int32 | Categorical grouping of items by name similarity; encoding weighted </br>by avg. strength of the group coupling (19 to 34420; roughly 2000 groups)
| item_category3_enc | int8 | reduction of original 84 categories, grouping primarily by item type |
| item_category4_enc | int8 | reduction of original 84 categories, grouping primarily by item brand |




**Code in this ipynb does the following:**

***sales_train*** dataset outliers: 

* Delete these rows (use this order for deleting if using .iloc): [2909818,2909401,2326930,2257299,1163158,484683]


***sales_train*** dataset shop overlap / untested shops: 

* Combine shop 11 into shop 10 (i.e., wherever you see shop_id == 11, set it to shop_id = 10), so *sales_train* no longer contains any shop 11.
* Combine shop 0 into shop 57 (id == 0 --> set id = 57)
* Combine shop 1 into shop 58 (id == 1 --> set id = 58)
* Delete all *sales_train* rows where shop_id == 9
* Delete all *sales_train* rows where shop_id == 13

***shops_new*** dataset features:

* continue using ***shop_id*** as a categorical feature for training/test; suggest encoding it by weighting with the number of rows present in the ***test set***
* add ***shop_type*** as a categorical feature (also recommend encoding these categories by weighting with the number of rows present in ***test*** set)
* also see if ***shop_federal_district*** has significant feature importance (also encode based on relative presence in ***test*** set), depending on your time/computing resources.  I will also add ***shop_city*** as another potential feature if you have time/computing resources to try it.
* I'm adding 3 other potential features (2 columns: one for text description, and one for integer encoding):  
  * s_type_broad (like shop_type, but fewer categories)
  * fd_popdens (4-category grouping based on shop federal district population density)
  * fd_gdp (3-category grouping based on gdp per person in the shop's federal district)

***items_new*** dataset features:

* delete ***item_name*** column (just a waste of space for training purposes)
* merge with ***item_categories*** dataset to add ***item_category3*** and ***item_category4*** as feature columns.  Encoding randomly done by pandas.
* merge with ***items_clustered_22170*** dataset to add ***item_cluster_code*** as a pre-encoded categorical feature.
* suggest using ***item_id***, ***item_cluster_code***, ***item_category3_enc***, ***item_category_id***, and ***item_category4_enc*** as features in the model, in that order of importance (depending on your time/computing resources).  

</br>

</br>

**Some Thoughts On The Cleaning and Use of Features Above:**

1.  **Category overlap** in the new features of the items dataset:</br>
If one or more of these 4 category/cluster groupings shows little feature importance upon training, then drop it from the model inputs.  If one or two of the 4 groupings are substantially higher importance upon training, then suggest you do not use the other 2 or 3 category groupings as features at all. (There is *some* correlation between the categories of two different grouping methods, but I would be concerned that there are also anti-correlations between the different feature variants, and that it is not straightforward for the model to resolve conflicting category information.  I'm guessing that the best results would come from using one or two of the "broad" category groupings (cat3 or cat4) and one (and only one) of the "finer" category groupings (the original 84-category feature, or the weighted 2000ish-category cluster_code feature)

2.  **Shops inclusion and feature priority**:</br>
* I suspect *shop_federal_district* will not have a great importance in the training, but I would be interested to see if you find the same conclusion after training and looking at relative feature importance.  Although I'm also including *shop_city* as a possible feature to use, I don't recommend this unless you have plenty of time/computing resources.  If you can, give it a try, and see if it actually does have significant feature importance.
* I am dropping shops 9 and 13 from the training set due to their odd behavior and because they are not in the test set.  I don't think there will be worse model performance because of this (I suspect actually it could be significantly better), but if time permits, it would be worth double-checking this hypothesis by training the model with these two shops still in the *sales_train* data set, and see if you see any model improvement.
* Similarly, the merging / removal of shops 0, 1, and 11 is being done on a hunch that the similarity of these 3 untested shops with their (tested) merge partners is so strong that the merging to create more data for training tested shops will be beneficial.  The issue is that I have not done a deep dive into the aforementioned similarity.  There is a possibility that this merging of shops could distort the training behavior of time lag, item_id, or item_category if the shop 'similiarity' is not strong enough.  For example,  the 3 shop merges could create a substantially different sales distribution of items/categories/time for the 3 shops being tested. A shallow exploration suggests it's not an issue, but I'm not 100% confident, and we should consider poentially running the model training without merging these shops together.  (The good news is I don't think the 3 merges will have much effect, if any, on any of the other shops.)



#0.1 Configure Environment
**NOT OPTIONAL**

In [ ]:
# General python libraries/modules used throughout the notebook
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import numpy as np
from scipy import sparse
import seaborn as sns
from numba import jit, prange
import networkx as nx
from networkx.algorithms import community, cluster

import os
from itertools import product
import re
import json
import time
from time import sleep, localtime, strftime
import pickle
from collections import defaultdict


# Magics
%matplotlib inline


# NLP packages
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

# # ML packages
# from sklearn.linear_model import LinearRegression

# !pip install catboost
# from catboost import CatBoostRegressor 

# %tensorflow_version 2.x
# import tensorflow as tf
# import keras as K

# # List of the modules we need to version-track for reference
modules = ['pandas','matplotlib','numpy','scipy','numba','seaborn','sklearn','tensorflow','keras','catboost','pip','nltk','networkx']

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Notebook formatting
# Adjust as per your preferences.  I'm using a FHD monitor with a full-screen browser window containing my IPynb notebook

# format pandas output so we can see all the columns we care about (instead of "col1  col2  ........ col8 col9", we will see "col1 col2 col3 col4 col5 col6 col7 col8 col9" if it fits inside display.width parameter)
pd.set_option("display.max_columns",30)  
pd.set_option("display.max_rows",100)     # Override pandas choice of how many rows to show, so, for example, we can see the full 84-row item_category dataframe instead of the first few rows, then ...., then the last few rows
pd.set_option("display.width", 300)       # Similar to the above for showing more rows than pandas defaults to, we can show more columns than default, if we tune this to our monitor window size
pd.set_option("max_colwidth", None)

#pd.set_option("display.precision", 3)  # Nah, this is helpful, but below is even better
#Try to convince pandas to print without decimal places if a number is actually an integer (helps keep column width down, and highlights data types)
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.3f}'.format(x)

#0.2 Load Data Files



##0.2.1) Enter Data File Names and Paths

**NOT Optional**

In [ ]:
#  FYI, data is coming from a public repo on GitHub at github.com/migai/Kag
# List of the data files (path relative to GitHub master), to be loaded into pandas DataFrames
data_files = [  "readonly/final_project_data/items.csv",
                #"readonly/final_project_data/item_categories.csv",
                #"readonly/final_project_data/shops.csv",
                #"readonly/final_project_data/sample_submission.csv.gz",
                "readonly/final_project_data/sales_train.csv.gz",
                "readonly/final_project_data/test.csv.gz",
                #"data_output/shops_transl.csv",
                "data_output/shops_augmented.csv",
                #"data_output/item_categories_transl.csv",
                "data_output/item_categories_augmented.csv",
                #"data_output/items_transl.csv",
                "data_output/items_clustered_22170.csv.gz",
                #"readonly/en_50k.csv",
                "data_output/shops_new.csv",
                "data_output/items_new.csv",
                "data_output/sales_train_cleaned.csv.gz",
                "data_output/sales_train_cln_mrg.csv.gz",
                "data_output/test_mrg.csv.gz",
                "readonly/final_project_data/sample_submission.csv.gz",
                "models_and_predictions/LGBMv3_submission.csv"
              ]


# Dict of helper code files, to be loaded and imported {filepath : import_as}
code_files = {}  # not used at this time; example dict = {"helper_code/kaggle_utils_at_mg.py" : "kag_utils"}


# GitHub file location info
git_hub_url = "https://raw.githubusercontent.com/migai"
repo_name = 'Kag'
branch_name = 'master'
base_url = os.path.join(git_hub_url, repo_name, branch_name)

##0.2.2) Load Data Files

In [ ]:
# click on the URL link presented to you by this command, get your authorization code from Google, then paste it into the input box and hit 'enter' to complete mounting of the drive
from google.colab import drive  
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
############################################################
############################################################
'''
# Replace this path with the path on *your* Google Drive where the repo master branch is stored
#   (on GitHub, the remote repo is located at github.com/migai/Kag --> below is my cloned repo location)
GDRIVE_REPO_PATH = "/content/drive/My Drive/Colab Notebooks/NRUHSE_2_Kaggle_Coursera/final/Kag"
'''
############################################################
############################################################
'''

%cd "{GDRIVE_REPO_PATH}"

print("Loading Files from Google Drive repo into Colab...\n")

# Loop to load the data files into appropriately-named pandas DataFrames
for path_name in data_files:
    filename = path_name.rsplit("/")[-1]
    data_frame_name = filename.split(".")[0]
    exec(data_frame_name + " = pd.read_csv(path_name)")
    if data_frame_name == 'sales_train':
        sales_train['date'] = pd.to_datetime(sales_train['date'], format = '%d.%m.%Y')
    print("Data Frame: " + data_frame_name)
    print(eval(data_frame_name).head(2))
    print("\n")


/content/drive/My Drive/Colab Notebooks/NRUHSE_2_Kaggle_Coursera/final/Kag
Loading Files from Google Drive repo into Colab...

Data Frame: items
                                                              item_name  item_id  item_category_id
0                             ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0                40
1  !ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]        1                76


Data Frame: sales_train
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-01-02               0       59    22154         999             1
1 2013-01-03               0       25     2552         899             1


Data Frame: test
   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320


Data Frame: shops_augmented
                       shop_name  shop_id                       en_shop_name shop_city shop_category shop_federal_district  shop_city_population  shop_tested
0  !Якутск Орджоникидзе, 56 фран  

In [ ]:
df_name_dict = {'shops_new':shops_new,'items_new':items_new,'sales_train_cleaned':sales_train_cleaned,'sales_train_cln_mrg':sales_train_cln_mrg,'test_mrg':test_mrg}

for k,v in df_name_dict.items():
    print(f'{k}: n_rows = {len(v)}, n_cols = {len(v.columns)}\n{v.columns}\n')

shops_new: n_rows = 60, n_cols = 14
Index(['shop_id', 'shop_tested', 'shop_type', 'shop_type_enc', 'shop_city', 'shop_city_enc', 'shop_federal_district', 'shop_federal_district_enc', 's_type_broad', 's_type_broad_enc', 'fd_popdens', 'fd_popdens_enc', 'fd_gdp', 'fd_gdp_enc'], dtype='object')

items_new: n_rows = 22170, n_cols = 8
Index(['item_id', 'item_tested', 'item_category_id', 'cluster_code', 'item_category3', 'item_category3_enc', 'item_category4', 'item_category4_enc'], dtype='object')

sales_train_cleaned: n_rows = 2914268, n_cols = 6
Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day'], dtype='object')

sales_train_cln_mrg: n_rows = 2914268, n_cols = 18
Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day', 'shop_tested', 'shop_type_enc', 'shop_city_enc', 'shop_federal_district_enc', 's_type_broad_enc', 'fd_popdens_enc', 'fd_gdp_enc', 'item_tested', 'item_category_id', 'cluster_code', 'item_category3_enc',
       'it

In [ ]:
print(len(sample_submission))
sample_submission.head()

214200


,ID,item_cnt_month
0,0,0.500
1,1,0.500
2,2,0.500
3,3,0.500
4,4,0.500


In [ ]:
print(len(LGBMv3_submission))
LGBMv3_submission.head()

214200


,ID,item_cnt_month
0,0,0.323
1,1,0.132
2,2,0.661
3,3,0.306
4,4,1.714


#1. Create ***shops_new*** data file </br>
**60 rows**, corresponding to the 60 original shops</br>
**14 columns:**
 * shop_id  (categorical; 0-59, int8, from original data set)
 * shop_tested (categorical; bool, indicating if the shop is in *test* set)
 * shop_type (categorical; string object: online, small shop, mall, SEC, Mega)
 * shop_type_enc (categorical; int8, ordinal/weighted encoding based on number of rows present in *test*, scaled to cover roughly the same range as shop_id values (0-59))
 * shop_city (categorical; string)
 * shop_city_enc (categorical; int8, encoding weighted like shop_type)
 * shop_federal_district (categorical; string object)
 * shop_federal_district_enc (categorical; int8, encoding weighted like shop_type)
 * s_type_broad (categorical; string object: like shop_type, but fewer categories by merging together "Mall","Mega","SEC")
 * s_type_broad_enc (categorical; int8, ordinal encoding roughly weighted by shop size, 0-60 scale)
 * fd_popdens (categorical; string object: 4 categories named by population density in the shop's federal district)
 * fd_popdens_enc (categorical; int8; ordinal encoding weight based on population density)
 * fd_gdp (categorical; string object: 3 categories named by gdp per person)
 * fd_gdp_enc (categorical; int8; ordinal encoding weight based on gdp/person)

In [ ]:
# each of the shops in the test set has 5100 rows in the test set, but not all shops are present in the test set
# encode categories in the new dataset such that there is more weight given to the category values with more presence in the test set
shops_new = shops_augmented[['shop_id','shop_category','shop_city','shop_federal_district','shop_tested']].rename(columns={'shop_category':'shop_type'})
shops_new['test_rows'] = shops_new.shop_tested.apply(lambda x: 5100 if x else 0)
types = defaultdict.fromkeys(shops_new.shop_type.unique(),0)
cities = defaultdict.fromkeys(shops_new.shop_city.unique(),0)
feddists = defaultdict.fromkeys(shops_new.shop_federal_district.unique(),0)
for i in range(len(shops_new)):  # most of the weighting used to order these categories comes from the number of test rows; number of train rows helps to break ties
    train_items_sold = int(round(sales_train[sales_train.shop_id == i].item_cnt_day.sum()/1e3))
    types[shops_new.at[i,'shop_type']] += shops_new.at[i,'test_rows']*50 + train_items_sold
    cities[shops_new.at[i,'shop_city']] += shops_new.at[i,'test_rows']*50 + train_items_sold
    feddists[shops_new.at[i,'shop_federal_district']] += shops_new.at[i,'test_rows']*50 + train_items_sold

enc_types = defaultdict(np.int8)
enc_cities = defaultdict(np.int8)
enc_feddists = defaultdict(np.int8)
for feat in [[types,enc_types],[cities,enc_cities],[feddists,enc_feddists]]:
    enc = 0
    for cat in sorted(feat[0], key=feat[0].get): #, reverse=True):
        feat[1][cat] = enc * int(round((60 / len(feat[0]))))  # scale encoding to be similar to 0-60 range of shop_id
        enc+=1

shops_new['shop_type_enc'] = shops_new.shop_type.apply(lambda x: enc_types[x])
shops_new['shop_city_enc'] = shops_new.shop_city.apply(lambda x: enc_cities[x])
shops_new['shop_federal_district_enc'] = shops_new.shop_federal_district.apply(lambda x: enc_feddists[x])


# # From Wikipedia (2010, 2014, and 2017 numbers)

# Federal district	Population density(per km2)	GDP per capita (2017)
# Central	          59                          $11423
# Northwestern        8                           $10088
# Southern            33                          $5592
# North Caucasian     55                          $3262
# Volga               29                          $6388
# Ural                7                           $14819
# Siberian            4                           $6887
# Far Eastern         1                           $10767
# popdensity categories = remote, intermediate, populous; encode values as: (1+4+7+8)/4 = 5, (29+33)/2 = 31, (55+59)/2 = 57, online = overall avg = 196/8 = 24.5 --> 25
# gdp categories = bottom(3262), low((5592+6388+6887)/3)=6289), intermediate((10767+10088+11423)/3) = 10759), high(14819) --> use "high" for online; 
#      divide all by 250 to scale closer to shop_id encoding range --> 13, 25, 43, 60

type2cats_encs = {'featurename':'s_type_broad','enc_name':'s_type_broad_enc','Shop': ['Shop',10], 'Mall': ['Mall',60], 'Mega': ['Mall',60], 'SEC': ['Mall',60], 'Itinerant': ['Online',35], 'Online': ['Online',35]}
popdensitycats_encs = {'featurename':'fd_popdens','enc_name':'fd_popdens_enc','Eastern': ['Remote',5], 'South': ['Intermediate',31], 'Central': ['Populous',57], 'Northwestern': ['Remote',5], 
                  'None': ['Online',25], 'Volga': ['Intermediate',31], 'Siberian': ['Remote',5], 'Ural': ['Remote',5]}
gdpcats_encs = {'featurename':'fd_gdp','enc_name':'fd_gdp_enc','Eastern': ['Intermediate',43], 'South': ['Low',25], 'Central': ['Intermediate',43], 'Northwestern': ['Intermediate',43], 
                  'None': ['High',60], 'Volga': ['Low',25], 'Siberian': ['Low',25], 'Ural': ['High',60]}

for feat in [type2cats_encs]:
    shops_new[feat['featurename']] = shops_new.shop_type.apply(lambda x: feat[x][0])
    shops_new[feat['enc_name']] = shops_new.shop_type.apply(lambda x: feat[x][1])

for feat in [popdensitycats_encs,gdpcats_encs]:
    shops_new[feat['featurename']] = shops_new.shop_federal_district.apply(lambda x: feat[x][0])
    shops_new[feat['enc_name']] = shops_new.shop_federal_district.apply(lambda x: feat[x][1])

shops_new.drop('test_rows',axis=1,inplace=True)
shops_new = shops_new[['shop_id','shop_tested','shop_type','shop_type_enc','shop_city','shop_city_enc','shop_federal_district',
                       'shop_federal_district_enc','s_type_broad','s_type_broad_enc','fd_popdens','fd_popdens_enc','fd_gdp','fd_gdp_enc']]

shops_new = shops_new.astype({'shop_id':np.int8,'shop_tested':'bool','shop_type':'str','shop_type_enc':np.int8,'shop_city':'str',
                              'shop_city_enc':np.int8,'shop_federal_district':'str','shop_federal_district_enc':np.int8,
                              's_type_broad':'str','s_type_broad_enc':np.int8,'fd_popdens':'str','fd_popdens_enc':np.int8,'fd_gdp':'str','fd_gdp_enc':np.int8})
print('\n',shops_new.dtypes)
print('\n',shops_new)

# shops_new.to_csv("data_output/shops_new.csv", index=False)


 shop_id                        int8
shop_tested                    bool
shop_type                    object
shop_type_enc                  int8
shop_city                    object
shop_city_enc                  int8
shop_federal_district        object
shop_federal_district_enc      int8
s_type_broad                 object
s_type_broad_enc               int8
fd_popdens                   object
fd_popdens_enc                 int8
fd_gdp                       object
fd_gdp_enc                     int8
dtype: object

     shop_id  shop_tested  shop_type  shop_type_enc        shop_city  shop_city_enc shop_federal_district  shop_federal_district_enc s_type_broad  s_type_broad_enc    fd_popdens  fd_popdens_enc        fd_gdp  fd_gdp_enc
0         0        False       Shop             20          Yakutsk             54               Eastern                         16         Shop                10        Remote               5  Intermediate          43
1         1        False       Mall     

#2. Create ***items_new*** data file </br>
**22,170 rows**, corresponding to the 22,170 original items</br>
**7 columns:**
 * item_id  (categorical; range(22170), int16, from original data set)
 * item_tested (categorical; bool, indicating if item is in *test* set)
 * item_category_id (categorical; range(84), int8, from original data set)
 * cluster_code (categorical; int32, weighted encoding based on similarity of item names in a given cluster)
 * item_category3 (categorical; string)
 * item_category3_enc (categorical; int8, random nominal encoding done by pandas)
 * item_category4 (categorical; string)
 * item_category4_enc (categorical; int8, random nominal encoding done by pandas)


In [ ]:

item_categories_augmented["item_category3"] = item_categories_augmented["item_category3"].astype('category')
item_categories_augmented["item_category3_enc"] = item_categories_augmented['item_category3'].cat.codes
item_categories_augmented["item_category4"] = item_categories_augmented["item_category4"].astype('category')
item_categories_augmented["item_category4_enc"] = item_categories_augmented['item_category4'].cat.codes

items_new = items_clustered_22170[['item_id','item_tested','item_category_id','cluster_code']].copy(deep=True)

items_new = items_new.merge(item_categories_augmented[['item_category_id','item_category3','item_category3_enc','item_category4','item_category4_enc']],how='left',on='item_category_id')
# print(len(items_new))
# print(items_new.head())
# print(items_new['item_category3_enc'].unique())
# print(items_new['item_category4_enc'].unique())
items_new = items_new.astype({'item_id':np.int16,'item_tested':'bool','item_category_id':np.int8,'cluster_code':np.int32,
                              'item_category3':'str','item_category3_enc':np.int8, 'item_category4':'str','item_category4_enc':np.int8}) 

print('\n',items_new.dtypes)
print('\n',items_new.head(20))

# items_new.to_csv("data_output/items_new.csv", index=False)


 item_id                int16
item_tested             bool
item_category_id        int8
cluster_code           int32
item_category3        object
item_category3_enc      int8
item_category4        object
item_category4_enc      int8
dtype: object

     item_id  item_tested  item_category_id  cluster_code item_category3  item_category3_enc item_category4  item_category4_enc
0         0        False                40           920         Movies                   7         Movies                   3
1         1        False                76          2600       Software                  10             PC                   5
2         2        False                40           802         Movies                   7         Movies                   3
3         3        False                40           330         Movies                   7         Movies                   3
4         4        False                40          1686         Movies                   7         Movies         

#3. Create ***sales_train_cleaned*** data file 
Remove outliers, shops 9 and 13, and merge shops 0,1,11 (and downcast)

</br>

**2,935,849 - 6 = 2,935,843 rows**, corresponding to the original rows ***with outliers removed*** (6 outlier rows deleted)=

* Delete these rows (use this order for deleting if using .iloc): [2909818,2909401,2326930,2257299,1163158,484683]

</br>

**2,914,268 rows**, after removal and merging of the shops:

* Combine shop 11 into shop 10 (i.e., wherever you see shop_id == 11, set it to shop_id = 10), so *sales_train* no longer contains any shop 11.
* Combine shop 0 into shop 57 (id == 0 --> set id = 57)
* Combine shop 1 into shop 58 (id == 1 --> set id = 58)
* Delete all *sales_train* rows where shop_id == 9
* Delete all *sales_train* rows where shop_id == 13

</br>

**6 columns:**
 * date (pd.datetime format)
 * date_block_num (int8) 
 * shop_id (categorical; int8, 55 unique values, inside the range 2 to 59, having removed shops 9 and 13, and merged shops 0,1,11) 
 * item_id (categorical; int16, range 0 to 21,699)
 * item_price (float32; max is near 60,000, roughly 4500 items 0 < $p <= 0.5, so don't round off to integer values)  
 * item_cnt_day (int16, range is roughly 0 to 1000)


In [ ]:
sales_train_cleaned = sales_train.copy(deep=True)
print(len(sales_train_cleaned))
for i in [2909818,2909401,2326930,2257299,1163158,484683]:
    print(sales_train_cleaned[sales_train_cleaned.index == i])
    sales_train_cleaned.drop(sales_train_cleaned.index[i],inplace=True)
print(len(sales_train_cleaned))

print(sales_train_cleaned.shop_id.nunique())

sales_train_cleaned = sales_train_cleaned[sales_train_cleaned.shop_id != 9]
print(len(sales_train_cleaned))
sales_train_cleaned = sales_train_cleaned[sales_train_cleaned.shop_id != 13]
print(len(sales_train_cleaned))

sales_train_cleaned['shop_id'] = sales_train_cleaned.shop_id.apply(lambda x: 57 if x == 0 else x)
sales_train_cleaned['shop_id'] = sales_train_cleaned.shop_id.apply(lambda x: 58 if x == 1 else x)
sales_train_cleaned['shop_id'] = sales_train_cleaned.shop_id.apply(lambda x: 10 if x == 11 else x)


sales_train_cleaned = sales_train_cleaned.astype({'date_block_num':np.int8,'shop_id':np.int8,'item_id':np.int16,
                              'item_price':np.float32,'item_cnt_day':np.int16}) 

print('\n',sales_train_cleaned.dtypes)
print('\n',sales_train_cleaned.head())

compression_opts = dict(method='gzip',
                        archive_name='sales_train_cleaned.csv')  
sales_train_cleaned.to_csv('data_output/sales_train_cleaned.csv.gz', index=False, compression=compression_opts)

2935849
              date  date_block_num  shop_id  item_id  item_price  item_cnt_day
2909818 2015-10-28              33       12    11373       0.909          2169
              date  date_block_num  shop_id  item_id  item_price  item_cnt_day
2909401 2015-10-14              33       12    20949           4           500
              date  date_block_num  shop_id  item_id  item_price  item_cnt_day
2326930 2015-01-15              24       12    20949           4          1000
              date  date_block_num  shop_id  item_id  item_price  item_cnt_day
2257299 2014-12-19              23       12    20949           4           500
              date  date_block_num  shop_id  item_id  item_price  item_cnt_day
1163158 2013-12-13              11       12     6066      307980             1
             date  date_block_num  shop_id  item_id  item_price  item_cnt_day
484683 2013-05-15               4       32     2973          -1             1
2935843
60
2932092
2914268

 date             

#4. Create ***sales_train_cln_mrg*** and ***test_mrg*** data files </br>
Remove outliers, adjust shops, merge with several encoded features from *shops_new* and *items_new*

</br>

##Merge overview: (how = "left")
left = *sales_train_cleaned* dataframe and/or *test* dataframe

right = *shops_new* on 'shop_id'

right = *items_new* on 'item_id'

***only merge select columns, to keep sales_train_cln_mrg filesize manageable***

</br>

###***sales_train_cleaned*** dataframe:

**2,914,268 rows** (down from original 2,935,849 by removing outliers, shop 9, shop 13)

**6 columns:**

| dtype: | datetime64[ns] | int8 | int8 | int16 | float32 | int16 |
| :----: | :----: | :----: | :----: | :----: | :----: | :----: |
| row | date | date_block_num | shop_id | item_id | item_price | item_cnt_day |
| 0 | 2013-01-02 | 0 | 59 | 22154 | 999 | 1 |
| 1 | 2013-01-03 | 0 | 25 | 2552  | 899 | 1 |
| 2 | 2013-01-05 | 0 | 25 | 2552  | 899 | -1 |
| 3 | 2013-01-06 | 0 | 25 | 2554  | 1709 | 1 |
| 4 | 2013-01-15 | 0 | 25 | 2555  | 1099 | 1 |

</br>

###***shops_new*** dataframe:
**60 rows**, corresponding to the 60 original shops</br>
**14 columns** -- we will merge the bold columns below, on = 'shop_id'
 * shop_id  (categorical; 0-59, int8, from original data set)
 * ***shop_tested*** (categorical; bool, indicating if the shop is in *test* set)
 * shop_type (categorical; string object: online, small shop, mall, SEC, Mega)
 * ***shop_type_enc*** (categorical; int8, ordinal/weighted encoding based on number of rows present in *test*, scaled to cover roughly the same range as shop_id values (0-59))
 * shop_city (categorical; string)
 * ***shop_city_enc*** (categorical; int8, encoding weighted like shop_type)
 * shop_federal_district (categorical; string object)
 * ***shop_federal_district_enc*** (categorical; int8, encoding weighted like shop_type)
 * s_type_broad (categorical; string object: like shop_type, but fewer categories by merging together "Mall","Mega","SEC")
 * ***s_type_broad_enc*** (categorical; int8, ordinal encoding roughly weighted by shop size, 0-60 scale)
 * fd_popdens (categorical; string object: 4 categories named by population density in the shop's federal district)
 * ***fd_popdens_enc*** (categorical; int8; ordinal encoding weight based on population density)
 * fd_gdp (categorical; string object: 3 categories named by gdp per person)
 * ***fd_gdp_enc*** (categorical; int8; ordinal encoding weight based on gdp/person)

</br>

###***items_new*** dataframe:

**22,170 rows**, corresponding to the 22,170 original items</br>
**7 columns** -- we will merge the 5 bold columns below, on = 'item_id'
 * item_id  (categorical; 0 - 22169, int16, from original data set)
 * ***item_tested*** (categorical; bool, indicating if item is in *test* set)
 * ***item_category_id*** (categorical; 0-83, int8, from original data set)
 * ***cluster_code*** (categorical; int32, weighted encoding based on similarity of item names in a given cluster)
 * item_category3 (categorical; string)
 * ***item_category3_enc*** (categorical; int8, random nominal encoding done by pandas)
 * item_category4 (categorical; string)
 * ***item_category4_enc*** (categorical; int8, random nominal encoding done by pandas)


In [ ]:
print(f'Number of rows in sales_train: {len(sales_train)}')
print(f'Number of rows in sales_train_cleaned: {len(sales_train_cleaned)}')
print(f'Number of columns in sales_train_cleaned: {len(sales_train_cleaned.columns)}')
print(f'Column datatypes for sales_train_cleaned:\n{sales_train_cleaned.dtypes}')
print(f'\nFirst 2 rows of sales_train_cleaned:\n{sales_train_cleaned.head(2)}')
print('\n')
print(f'Number of rows in test: {len(test)}')
print(f'Number of columns in test: {len(test.columns)}')
print(f'Column datatypes for test:\n{test.dtypes}')
print(f'\nFirst 2 rows of test:\n{test.head(2)}')
print('\n')

# Merge shop category encodings
sales_train_cln_mrg = sales_train_cleaned.merge(shops_new[['shop_id','shop_tested','shop_type_enc','shop_city_enc','shop_federal_district_enc',
                                                           's_type_broad_enc','fd_popdens_enc','fd_gdp_enc']], how='left', on='shop_id')
test_mrg = test.merge(shops_new[['shop_id','shop_tested','shop_type_enc','shop_city_enc','shop_federal_district_enc',
                                                           's_type_broad_enc','fd_popdens_enc','fd_gdp_enc']], how='left', on='shop_id')

# Merge item category encodings
sales_train_cln_mrg = sales_train_cln_mrg.merge(items_new[['item_id','item_tested','item_category_id','cluster_code',
                                                           'item_category3_enc','item_category4_enc']], how='left', on='item_id')
test_mrg = test_mrg.merge(items_new[['item_id','item_tested','item_category_id','cluster_code',
                                                           'item_category3_enc','item_category4_enc']], how='left', on='item_id')

# Reduce size of test_mrg columns from int64 to 32 or 16 or 8
test_mrg = test_mrg.astype({'ID':np.int32,'shop_id':np.int8,'item_id':np.int16}) 



print(f'Number of rows in sales_train_cln_mrg: {len(sales_train_cln_mrg)}')
print(f'Number of columns in sales_train_cln_mrg: {len(sales_train_cln_mrg.columns)}')
print(f'Column datatypes for sales_train_cln_mrg:\n{sales_train_cln_mrg.dtypes}')
print(f'\nFirst 2 rows of sales_train_cln_mrg:\n{sales_train_cln_mrg.head(2)}')
print('\n')
print(f'Number of rows in test_mrg: {len(test_mrg)}')
print(f'Number of columns in test_mrg: {len(test_mrg.columns)}')
print(f'Column datatypes for test_mrg:\n{test_mrg.dtypes}')
print(f'\nFirst 2 rows of test_mrg:\n{test_mrg.head(2)}')


Number of rows in sales_train: 2935849
Number of rows in sales_train_cleaned: 2914268
Number of columns in sales_train_cleaned: 6
Column datatypes for sales_train_cleaned:
date              datetime64[ns]
date_block_num              int8
shop_id                     int8
item_id                    int16
item_price               float32
item_cnt_day               int16
dtype: object

First 2 rows of sales_train_cleaned:
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-01-02               0       59    22154         999             1
1 2013-01-03               0       25     2552         899             1


Number of rows in test: 214200
Number of columns in test: 3
Column datatypes for test:
ID         int64
shop_id    int64
item_id    int64
dtype: object

First 2 rows of test:
   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320


Number of rows in sales_train_cln_mrg: 2914268
Number of columns in sales_train_cln_mrg: 18
Column datatypes

In [ ]:
# save the gzipped sales_train_cln_mrg and test_mrg dataframes to GoogleDrive (& GitHub)

compression_opts = dict(method='gzip',
                        archive_name='sales_train_cln_mrg.csv')  
sales_train_cln_mrg.to_csv('data_output/sales_train_cln_mrg.csv.gz', index=False, compression=compression_opts)

compression_opts = dict(method='gzip',
                        archive_name='test_mrg.csv')  
test_mrg.to_csv('data_output/test_mrg.csv.gz', index=False, compression=compression_opts)

In [ ]:
print(sales_train_cln_mrg['item_id'].nunique(), sales_train_cln_mrg['shop_id'].nunique())
print(len(items_new))

21671 55
22170
22170


In [ ]:
print(sales_train_cln_mrg.item_price.max(), sales_train_cln_mrg.item_price.min())
print(sales_train_cln_mrg.item_cnt_day.max(), sales_train_cln_mrg.item_cnt_day.min())
print(sales_train_cln_mrg.cluster_code.max(), sales_train_cln_mrg.cluster_code.min())

59200 0
669 -22
34420 19


In [ ]:
print(len(sales_train_cln_mrg[sales_train_cln_mrg.item_price ==0]))
print(len(sales_train[sales_train.item_price < 1]))
print(len(sales_train[sales_train.item_price <= 0.5]))
st9_0 = sales_train[sales_train.shop_id == 9]
print(len(st9_0[st9_0.item_price < 1]))
st13_0 = sales_train[sales_train.shop_id == 13]
print(len(st13_0[st13_0.item_price < 1]))
st_rnd = sales_train.item_price.apply(lambda x: int(round(x)))
print(len(st_rnd[st_rnd == 0]))

4163
4658
4164
0
0
4163
